# Load csv file

Let's start with getting the datafiles loan.csv.

In [2]:
# Import the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# To avoid DtypeWarning of column 47, mention low_memory=False
loan = pd.read_csv('loan.csv', low_memory=False)
loan.head()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
# Print the structure of the data, it seems because of 111 columns, info() is not displaying
print(loan.info(), "\n")
print(loan.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB
None 

(39717, 111)


In [5]:
# identify the unique number of id
print (len(loan.id.unique()))
print (len(loan.member_id.unique()))

39717
39717


In [6]:
# Summing up the missing values (column-wise) and displaying fraction of NaNs
# It seems we have so many columns with 100% missing values
round(100*(loan.isnull().sum()/len(loan.index)), 2)

id                              0.0
member_id                       0.0
loan_amnt                       0.0
funded_amnt                     0.0
funded_amnt_inv                 0.0
                              ...  
tax_liens                       0.1
tot_hi_cred_lim               100.0
total_bal_ex_mort             100.0
total_bc_limit                100.0
total_il_high_credit_limit    100.0
Length: 111, dtype: float64

# Data Cleansing

 Fixing Rows and Columns: DataType changes and column values fixing

In [11]:
# Convert string object to date object for below columns
loan['issue_d'] = pd.to_datetime(loan['issue_d'], format = '%b-%y')
loan['last_pymnt_d'] = pd.to_datetime(loan['last_pymnt_d'], format = '%b-%y')
loan['last_credit_pull_d'] = pd.to_datetime(loan['last_credit_pull_d'], format = '%b-%y')
loan['issue_d']
# Remove % and change the datatype to float
loan['int_rate'] = loan['int_rate'].str.rstrip('%').astype('float')

0       2011-12-01
1       2011-12-01
2       2011-12-01
3       2011-12-01
4       2011-12-01
           ...    
39712   2007-07-01
39713   2007-07-01
39714   2007-07-01
39715   2007-07-01
39716   2007-06-01
Name: issue_d, Length: 39717, dtype: datetime64[ns]

In [16]:
# It shows there is extra space in column value
print (list(loan['term'])[0:5])
# Remove the space
loan['term'] = loan['term'].str.strip()
# Remove the months word from column value and change the datatype
loan['term'] = loan['term'].str.split().str[0].astype(int)
# Rename the column name
loan.rename(columns={'term': 'term_months'}, inplace=True)

['36 months', '60 months', '36 months', '36 months', '60 months']


# Missing values

Dropping unnecessary columns and columns with more % of missing values and filling missing values.

In [20]:
# Let's drop the columns with more than 30% missing values

thresh = len(loan)*0.7
thresh
loan.dropna(thresh = thresh, axis = 1, inplace = True)

In [18]:
# Print the shape of the dataframe
print(loan.shape)

# Summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(loan.isnull().sum()/len(loan.index)), 2)

(39717, 111)


id                              0.0
member_id                       0.0
loan_amnt                       0.0
funded_amnt                     0.0
funded_amnt_inv                 0.0
                              ...  
tax_liens                       0.1
tot_hi_cred_lim               100.0
total_bal_ex_mort             100.0
total_bc_limit                100.0
total_il_high_credit_limit    100.0
Length: 111, dtype: float64

Now, we have very less missing values for below columns:
emp_title, title,revol_util,last_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,chargeoff_within_12_mths, pub_rec_bankruptcies and tax_liens

In [ ]:
# Let's drop the below unnecessary columns by checking the DataFrame and Data Dictionary
loan = loan.drop(['id','member_id','pymnt_plan','emp_title','title','url','zip_code','last_credit_pull_d','out_prncp','out_prncp_inv','collections_12_mths_ex_med','earliest_cr_line','policy_code','initial_list_status','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','tax_liens'], axis=1)

In [21]:
# Print the shape of the dataframe
print(loan.shape)

# Summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(loan.isnull().sum()/len(loan.index)), 2)

(39717, 53)


id                            0.00
member_id                     0.00
loan_amnt                     0.00
funded_amnt                   0.00
funded_amnt_inv               0.00
term_months                   0.00
int_rate                      0.00
installment                   0.00
grade                         0.00
sub_grade                     0.00
emp_title                     6.19
emp_length                    2.71
home_ownership                0.00
annual_inc                    0.00
verification_status           0.00
issue_d                       0.00
loan_status                   0.00
pymnt_plan                    0.00
url                           0.00
purpose                       0.00
title                         0.03
zip_code                      0.00
addr_state                    0.00
dti                           0.00
delinq_2yrs                   0.00
earliest_cr_line              0.00
inq_last_6mths                0.00
open_acc                      0.00
pub_rec             

Let's impute the missing values for below columns:
revol_util, last_pymnt_d, pub_rec_bankruptcies

In [25]:
# Change the format from % to float
loan['revol_util'] = loan['revol_util'].str.rstrip('%').astype('float')/100

# summary stats of revol_util 
loan.revol_util.describe()

AttributeError: Can only use .str accessor with string values!

Since there is no much spread of data and the difference between mean and median is very small, let's impute the missing values with mean

In [ ]:
# Impute the revol_util column with mean value
loan.loc[np.isnan(loan['revol_util']),['revol_util']] = loan['revol_util'].mean()

In [ ]:
# summary stats of revol_util 
loan.pub_rec_bankruptcies.describe()

These are integer type variables and has 0,1,2 values, so we cannot impute the missing values with mode - the most common occurring value.

In [ ]:
# Convert to Category type
loan['pub_rec_bankruptcies'] = loan['pub_rec_bankruptcies'].astype('category')

# Display frequency of each category
loan['pub_rec_bankruptcies'].value_counts()

The most common value of pub_rec_bankruptcies is 0, so let's impute the missing values

In [ ]:
# Impute the pub_rec_bankruptcies column with mode value
loan.loc[pd.isnull(loan['pub_rec_bankruptcies']), ['pub_rec_bankruptcies']] = 0

In [ ]:
# It shows that the missing values occurres only for 'Charged Off' loan types
print (loan[loan['last_pymnt_d'].isnull()]['loan_status'].unique())

# Number of missing values
print (loan.last_pymnt_d.isnull().sum())

Let's fill the last_pymnt_d date by analysing monthly installment and total_pymnt.
installment is the mothly installment who has to pay for taken loan
total_pymnt is the payments received to date for total amount funded

So, total_pymnt/installment gives the number of months the borrower has paid which means "last_pymnt_d"

In [ ]:
# Create new derived metric to store the number of months the borrower has paid
loan['num_of_mnts_paid'] = round(loan['total_pymnt']/loan['installment'])
# Convert the datatype
loan['num_of_mnts_paid'] = loan['num_of_mnts_paid'].astype(int)
# Display subset data
loan.loc[loan['last_pymnt_d'].isnull(), ['funded_amnt','int_rate','installment','issue_d','last_pymnt_d','total_pymnt','num_of_mnts_paid']].head()

In [ ]:
# Create a column to store the last payment date which is created based on 'issue_d','num_of_mnts_paid columns
# Add +1 to date_of_mnts_paid as monthly installment will start from issue_d+1
loan['date_of_mnts_paid'] = loan.loc[loan['num_of_mnts_paid']!=0, ['issue_d','num_of_mnts_paid']].apply(lambda x: x['issue_d'] + pd.DateOffset(months = x['num_of_mnts_paid']+1), axis=1)


In [ ]:
# Fill the missing values of last_pymnt_d column from date_of_mnts_paid column
loan['last_pymnt_d'] = loan['last_pymnt_d'].fillna(loan['date_of_mnts_paid'])

# Drop the date_of_mnts_paid column which was created for filling missing values
loan = loan.drop(['date_of_mnts_paid'], axis=1)

# Get the list of observations which were not filled the missing values
loan.loc[loan['last_pymnt_d'].isnull(), ['term_months','funded_amnt','int_rate','installment','issue_d','last_pymnt_d','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','num_of_mnts_paid']]

As the total_pymnt value is 0, we can't find the number of months, also there is no info about other columns like 'total_pymnt_inv','total_rec_prncp','total_rec_int' and as this is very minimal observations let's drop these 16 observations.

In [ ]:
# Drop the 16 observations of missing values of last_pymnt_d
loan = loan[pd.notnull(loan['last_pymnt_d'])]

# Summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(loan.isnull().sum()/len(loan.index)), 2)

In [ ]:
# Print the unique values of home_ownership
print (loan.home_ownership.unique())
# Check the number of loans by home_ownership
print (loan.home_ownership.value_counts())
    
# Replace the 'NONE' with 'RENT'(mode of home_ownership column)
loan['home_ownership'] = loan.home_ownership.str.replace('NONE','RENT')


In [ ]:
# Print the unique values of emp_length
print (loan.emp_length.unique())
# Check the number of loans by emp_length
print (loan.emp_length.value_counts())

# Replace the 'n/a' with '10+ years'(mode of emp_length column)
loan['emp_length'] = loan.emp_length.str.replace('< 1 year','0')
loan['emp_length'] = loan.emp_length.str.replace(' years','')
loan['emp_length'] = loan.emp_length.str.replace(' year','')
loan['emp_length'] = loan.emp_length.str.replace('+','')
loan['emp_length'] = loan.emp_length.str.replace('n/a','10')
# Convert datatype
loan['emp_length'] = pd.to_numeric(loan['emp_length'])


# Exract Type-driven Metrics


In [ ]:
# Extract the month, year from issue_d column
loan['issued_month'] = loan['issue_d'].dt.month
loan['issued_year'] = loan['issue_d'].dt.year

In [ ]:
# Derived Metric: Calculate the Return of Investment(ROI) by (total_pymnt-collection_recovery_fee/funded_amnt)-1
loan['roi'] = round(((loan['total_pymnt']-loan['collection_recovery_fee'])/loan['funded_amnt'])-1, 2)


In [ ]:
# Create the Segments for Annual Income
loan['income_segment'] = pd.cut(loan['annual_inc'],[3000,20000,40000,80000,150000,6000000],labels = ['lower','lower middle','middle','upper middle','upper'])
print (loan['income_segment'].unique())

# Create the Segments for Interest rate
loan['int_rate_segment'] = pd.cut(loan['int_rate'],[4,10,15,20,25],labels = ['5-10','11-15','16-20','21-25'])
print (loan['int_rate_segment'].unique())

In [ ]:
# For further analysis, let's filter the loan data by ignoring loan_status of 'Current', since these are currently running loans.
loan = loan[loan['loan_status']!='Current']

loan.info()


# Univariate Analysis on Unordered Categorical Variables


In [ ]:
# Univariate Analysis on Unordered Categorical variables
plt.figure(figsize=(15,10))

plt.figure(1)

# Distribution of Loans by Status
plt.subplot(3,2,1)
sns.countplot(x='loan_status', data=loan)
plt.title('Distribution of Loan Status')
plt.xlabel("Loan Status")
plt.ylabel("Number of Loans issued")

# Distribution of Loan by Purpose
plt.subplot(3,2,2)
sns.countplot(y='purpose', data=loan)
plt.title('Distribution of Loan by Purpose')
plt.xlabel("Number of Loans issued")
plt.ylabel("Loan Purpose")

# Distribution of Loan by State
plt.subplot(3,2,3)
sns.countplot(x='addr_state', data=loan)
plt.title('Distribution of Loan by State')
plt.xlabel("Number of Loans issued")
plt.ylabel("State")
# To rotate axis text
plt.xticks(rotation=90)

# Distribution of Loan by Home Ownership
plt.subplot(3,2,4)
sns.countplot(x='home_ownership', data=loan)
plt.title('Distribution of Loan by Home Ownership')
plt.xlabel("Home Ownership")
plt.ylabel("Number of Loans issued")

# Distribution of Loan by Term Month
plt.subplot(3,2,5)
sns.countplot(x='term_months', data=loan)
plt.title('Distribution of Loan by Term Month')
plt.xlabel("Term Month")
plt.ylabel("Number of Loans issued")

# Distribution of Loan by Income Verification Status
plt.subplot(3,2,6)
sns.countplot(x='verification_status', data=loan)
plt.title('Distribution of Loan by Income Verification Status')
plt.xlabel("Income Verification Status")
plt.ylabel("Number of Loans issued")

# To adjust spacing between subplots
plt.tight_layout()
plt.show()

Observation:

1) From this dataset, we have more observations(85%) from "Fully Paid" status.

2) There are more loan applicant's with purpose of debt consolidation.

3) There are more loan applicant's from California state.

4) Most of the loan applicant's are rented and mortgage.

5) There the more number of loan with 36 month term.

6) There the more number of loan which were not verified the annual income.

Borrowers with own house and want to consolidate debt are not at much risk, but borrower with rent,mortgage and want to consolidate debt are at high risk applicants.


# Univariate Analysis on Ordered Categorical Variables


In [ ]:
# Distribution of Loans by Grade and Sub-Grade
plt.figure(figsize=(12,6))

plt.figure(1)

plt.subplot(2,2,1)
sns.countplot( x="grade", data=loan,order = np.sort(loan['grade'].unique()))
plt.title("Distribution of Loans by Grade")
plt.xlabel("Grade")
plt.ylabel("Number of Loans issued")

plt.subplot(2,2,2)
sns.countplot( x="sub_grade", data=loan, order = np.sort(loan['sub_grade'].unique()))
plt.title("Distribution of Loans by Sub Grade")
plt.xlabel("Sub Grade")
plt.ylabel("Number of Loans issued")

# To rotate axis text
plt.xticks(rotation=90)


# Distribution of Loan Grades by Interest Rate
plt.subplot(2,2,3)
sns.boxplot(x="grade", y="int_rate", data=loan, order = np.sort(loan['grade'].unique()))
plt.title('Distribution of Loan Grades by Interest Rate')
plt.xlabel("Grade")
plt.ylabel("Interest Rate")

# Distribution of Funded Amount by Grade
plt.subplot(2,2,4)
sns.boxplot(x="grade", y="funded_amnt" , data=loan, order = np.sort(loan['grade'].unique()))
plt.title('Distribution of Funded Amount by Grade')
plt.xlabel("Grade")
plt.ylabel("Funded Amount")
plt.yscale('log')

# To adjust spacing between subplots
plt.tight_layout()

plt.show()

Observations:

1) From above plots, it shows that more number of loans were from B,A and C grade's and least from G grade.

2) From Sub grades A4, B3 have more number of loans.

3) From 3rd plot, it shows that A,B,C grade loans have less interest rate and E,F,G have high interest rate. From 1st, 2nd plots there are more number of loans from A,B,C grade(granularity check from sub-grades). It might be the reason that the loan applicant's from A,B,C grades have better credit score and lower risk.

4) From 4rt plot, it shows that there are high funded amount in A,B,C and D grades as the applicant's from these grades have better credit score and lower risk. 


In [ ]:
# Distribution of Loans by employment length
plt.figure(figsize=(12,6))
sns.countplot(x='emp_length', data=loan)
plt.title('Distribution of Loans by employment length')
plt.xlabel("Employment Length")
plt.ylabel("Number of Loans issued")
plt.show()

Observation:
1) The majority of borrowers have been employed for at least 10 years.

In [ ]:
# Number of loans issued (by year/month)
plt.figure(figsize=(12,6))

plt.figure(1)

plt.subplot(2,3,1)
sns.countplot( x="issued_month", data=loan[loan['issued_year']==2007])
plt.title("Loans by month of 2007 Year")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

plt.subplot(2,3,2)
sns.countplot( x="issued_month", data=loan[loan['issued_year']==2008])
plt.title("Loans by month of 2008 Year")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

plt.subplot(2,3,3)
sns.countplot( x="issued_month", data=loan[loan['issued_year']==2009])
plt.title("Loans by month of 2009 Year")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

plt.subplot(2,3,4)
sns.countplot( x="issued_month", data=loan[loan['issued_year']==2010])
plt.title("Loans by month of 2010 Year")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

plt.subplot(2,3,5)
sns.countplot( x="issued_month", data=loan[loan['issued_year']==2011])
plt.title("Loans by month of 2011 Year")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

plt.subplot(2,3,6)
sns.countplot( x="issued_month", data=loan)
plt.title("Loans by Month")
plt.xlabel("Month")
plt.ylabel("Number of Loans issued")

# To adjust spacing between subplots
plt.tight_layout()
plt.show()

Observations:

1) From above plots, it shows that more number of loans were issued in 11,12(November, December) months, the reasons could be Festivals(Thanks giving Day, Christmas and New Year).

2) In 2008 Year, there was huge spike in 3rd Month.

3) Number of loans issued increased steadily with slight descrease in 2008 year.


# Univariate Analysis on Quantitative Variables


In [ ]:
# Let's print descriptive data summary for all numerical columns
print (loan[['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment','annual_inc', 'dti', 'delinq_2yrs','emp_length', 'roi']].describe())
print ("\n")
print (loan[['inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util','total_acc',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int','total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
        'last_pymnt_amnt', 'application_type','pub_rec_bankruptcies', 'num_of_mnts_paid']].describe())

It seems all are fine except annual income(annual_inc). Let's analyse it separatly. 

In [ ]:
# It shows that the very high annual income exists after 75th percentile
loan['annual_inc'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
# Distribution of Annual income by employment length
plt.figure(figsize=(12,6))
sns.boxplot(x="emp_length", y="annual_inc" , data=loan)
plt.title('Distribution of Annual income by employment length')
plt.xlabel("Employment Length")
plt.ylabel("annual_inc")
plt.yscale('log')
plt.show()

Observations:

1) Almost all 'employment length' category has same level of 25th, 50th(Median) and 75th precentile's.


2) It clearly shows that there are high annual income values in almost all 'employment length' category. We can treat these as outliers.

In [ ]:
# Frequency distributions
plt.figure(figsize=(12,6))

plt.figure(1)

# Frequency distribution of DTI
plt.subplot(2,2,1)
sns.distplot(loan["dti"])

# Frequency distribution of revolving credit uilization
plt.subplot(2,2,2)
sns.distplot(loan["revol_util"])

# Frequency distribution of ration of open_acc to total_acc
plt.subplot(2,2,3)
sns.distplot(loan["open_acc"]/loan["total_acc"])
plt.xlabel("Ratio of open_acc to total_acc")

# Frequency distribution of ration of open_acc to total_acc
plt.subplot(2,2,4)
sns.distplot(loan["roi"])


# To adjust spacing between subplots
plt.tight_layout()
plt.show()

Observations:

1) The average debt-to-income ratio is 13%. There doesn’t seem to be much skew, considering the median is so close to the mean.

2) The mean of revolving credit utilization is 49%,  which means the average borrower is using most of their revolving credit at a time when they are seeking the loan. Also, there is large spread of data but not much skewed.

3) The percentage of open accounts to total accounts seems left skewed.

4) Negative ROI indicates defaulted loans while almost all of the loans with positive ROI were fully paid.

# Summary of Univariate Observations:

1) There are 40 variables (so far) with 39717 rows.

2) Number of loans issued increased steadily by every year with a slight decrease in 2008.

3) Of settled loans, 83% were Fully Paid and 14% were Charged Off.

4) Borrowers with own house and want to consolidate debt are not at much risk, but borrower with rent,mortgage and want to  consolidate debt are at high risk applicants.

5) Majority of loans were from A, B, and C grade.

6) There is an inverse relationship between interest rate and loan grade - lower grades have higher interest rate.



# Bivariate Analysis on Continuous Variables


In [ ]:
# Correlation Plot
cor = loan.corr()
round(cor,3)
plt.figure(figsize=(20,15))
sns.heatmap(cor, cmap='YlGnBu', annot=True)
plt.show()

From above correlation plot it show:

1) There are no highly negative correlation between columns.

2) There are some highly correlated columns at top and middle left block.

3) There is 0.45 correlation between term_months and interest rate(int_rate).

4) There is 0.47 correlation between revol_util and interest rate(int_rate).

5) There is no much correlation between Annual Income, DTI with other columns.

Let's check the bivariate distribution for below columns:
1) Interest rates vs ROI
2) Annual income vs ROI

In [ ]:
# ROI vs Interest Rate and Annual Income
plt.figure(figsize=(12,6))

sns.set(style="darkgrid", color_codes=True)

sns.jointplot(x="int_rate", y="roi", data=loan, kind="reg", color="r")

sns.jointplot(x="annual_inc", y="roi", data=loan, kind="reg", color="r")

plt.show()

Observations:

1) There was a positive correlation between interest rates and ROI for borrowers who paid their loans, but for those who didn’t pay the ROI was negative and it shows no correlation with interest rates.

2) Borrowers with higher income has positive ROI while borrowers with lower income has negative ROI.

In [ ]:
# Annual Income Vs Interest Rate distribution by each grade
plt.figure(figsize=(20,15))

sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='A'], hue='grade')
plt.yscale('log')


sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='B'], hue='grade')
plt.yscale('log')


sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='C'], hue='grade')
plt.yscale('log')


sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='D'], hue='grade')
plt.yscale('log')

sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='E'], hue='grade')
plt.yscale('log')

sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='F'], hue='grade')
plt.yscale('log')

sns.lmplot( x="int_rate", y="annual_inc", data=loan[loan['grade']=='G'], hue='grade')
plt.yscale('log')

plt.show()

Observations:

1) Grade A has borrowers with interest rate between 5-10%.

2) Grade B has borrowers with interest rate between 8-13%, but few are there with near 6%.

3) Grade C has borrowers with interest rate between 10-16%, but few are there with near 6%.

4) Grade D has borrowers with interest rate between 12-18%, but few are there with near 6%.

5) Grade E has borrowers with interest rate between 14-20%, but few are there with near 6%.

6) Grade F has borrowers with interest rate between 15-23%.

7) Grade G has borrowers with interest rate between 17-24%.

It is evident that the interest rate range changes from one grade to other, and also there are few borrowers with near 6% falling in B,C,D and E grade's.

In [ ]:
# Annual Income Vs DTI distribution by each grade
plt.figure(figsize=(20,15))

sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='A'], hue='grade')
plt.yscale('log')


sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='B'], hue='grade')
plt.yscale('log')


sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='C'], hue='grade')
plt.yscale('log')


sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='D'], hue='grade')
plt.yscale('log')

sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='E'], hue='grade')
plt.yscale('log')

sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='F'], hue='grade')
plt.yscale('log')

sns.lmplot( x="dti", y="annual_inc", data=loan[loan['grade']=='G'], hue='grade')
plt.yscale('log')

plt.show()

Observations:

1) It seems in A,B,C grades the data is spreaded and have DTI from 0-30.

2) Lower number of borrowers from 25-30 DTI in D,E grades.

3) Grade G has maximum 25 DTI.

It seems there is no DTI range constraint from one grade to other similar to interest rate which we have seen above.

# Bivariate Analysis on Categorical Variables




# ROI Bivariate Distribution plots


In [ ]:
# ROI vs Employment length by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="emp_length", y='roi', hue='loan_status',  data=loan)
plt.title('Distribution of ROI by Employment length')
plt.show()

Observations:

1) From fully_paid status loan, almost all employment length have same mean ROI.

2) From Charged_off status loan 1 years, 2 years and 7 years have low mean ROI.

In [ ]:
# ROI vs addr_state by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="addr_state", y='roi', hue='loan_status',  data=loan)
plt.title('Distribution of ROI by State')
# To rotate axis text
plt.xticks(rotation=90)
plt.show()

Observations:
  From Charged_off status loan UT,VT,MS states have low mean ROI.

In [ ]:
# ROI vs Grade by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="grade", y="roi" ,hue='loan_status', data=loan, order = np.sort(loan['grade'].unique()))
plt.title('Distribution of ROI by Grade')
plt.show()

Observations:

1) From fully_paid status loan, grades E,F,G have high mean ROI, this might be high interest rates.

2) From Charged_off status loan, grade G has low mean ROI.

In [ ]:
# ROI vs Loan Purpose by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="purpose", y='roi', hue='loan_status', data=loan, order=np.sort(loan['purpose'].unique()))
plt.title('Distribution of ROI by Loan Purpose')
# To rotate axis text
plt.xticks(rotation=90)
plt.show()


Observations:

1) From fully_paid status loan, almost all categories have nearby mean ROI.

2) From Charged_off status loan, small_business, other and medical have low mean ROI.

In [ ]:
# ROI vs Home Ownership by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="home_ownership", y='roi', hue='loan_status', data=loan, order=np.sort(loan['home_ownership'].unique()))
plt.title('Distribution of ROI by Home Ownership')
# To rotate axis text
plt.xticks(rotation=90)
plt.show()

Observations:
From Charged_off status loan, Rent and Mortgage have low mean ROI.

In [ ]:
# ROI vs Income Segment by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="income_segment", y='roi', hue='loan_status', data=loan, order=['lower','lower middle','middle','upper middle','upper'])
plt.title('Distribution of ROI by Income Segment')
plt.show()

Observations:

1) ROI is bit high for upper middle and upper Income_segment for fully paid loans.

2) ROI is very low for lower Income_segment for defaulted loans.

In [ ]:
# ROI vs Interest rate Segment by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="int_rate_segment", y='roi', hue='loan_status', data=loan, order=['5-10','11-15','16-20','21-25'])
plt.title('Distribution of ROI by Interest rate')
plt.show()

Observations:

1) It is evident that ROI is high when interest rate is high for paid loans.

2) On the other hand, ROI is low when interest rate is high for defaulted loans.




# Interest Rate Bivariate Distribution Plots


In [ ]:
# Interest Rate vs loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="loan_status", y='int_rate', data=loan)
plt.title('Distribution of Interest Rate by loan_status')
# To rotate axis text
plt.xticks(rotation=90)
plt.show()

Observations:
It shows that the interest rate is more from defaulters.

In [ ]:
# Interest Rate vs Term Months by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="term_months", y='int_rate', hue='loan_status',  data=loan)
plt.title('Distribution of Interest Rate by Term Months')
plt.show()

Observations:

1) It is evident that 60 months term loans have more interest rate.

2) There are more defaulters in both 36, 60 month terms because of high interest rates. 

In [ ]:
# Interest Rate vs Income Segment by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="income_segment", y='int_rate', hue='loan_status',  data=loan)
plt.title('Distribution of Interest Rate by Income Segment')
plt.show()

Observations:
It shows that, at every income segment, the defaulters are due to high interest rates.



# DTI Bivariate Distribution Plots


In [ ]:
# DTI vs Income Segment by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="income_segment", y='dti', hue='loan_status', data=loan)
plt.title('Distribution of DTI by Income Segment')
plt.show()

Observations:

1) DTI is less in lower segment, as the annual income is less and they might not have any loans and hence less DTI. If we have age details then we can predict more with borrower’s age, income and employment length.

2) DTI is high in lower middle and middle segments.

3) DTI is less in upper middle and upper segments, as the income is high they might have financial freedom. Note that there are some borrower’s in upper segment who have DTI more than 25.

In [ ]:
# DTI vs Loan Purpose by loan_status
plt.figure(num=None, figsize=(12,8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x="purpose", y='dti', hue='loan_status', data=loan, order=np.sort(loan['purpose'].unique()))
plt.title('Distribution of DTI by Loan Purpose')
# To rotate axis text
plt.xticks(rotation=90)
plt.show()

Observations:
Almost in all categories of purpose, defaulter's DTI is high than fully paid borrowers.



# Loan Defaulters Analysis 



In [ ]:
# Distribution of Loan Defaulters by Home Ownerships
sns.countplot(x='home_ownership',  data=loan[loan['loan_status']=='Charged Off'])
plt.title('Distribution of Loan Defaulters by Home Ownerships')
plt.xlabel("Home Ownerships")
plt.ylabel("Number of Loan Defaulters")
plt.show()

print (loan[loan['loan_status']=='Charged Off'].groupby(['home_ownership'])['home_ownership'].count())

Observation: It shows there are more defaulters in RENT and MORTGAGE. let's check it in granular level.

In [ ]:
#Distribution of Loan Defaulters by Loan Purpose
sns.countplot(x='purpose',  data=loan[loan['loan_status']=='Charged Off'], order=np.sort(loan['purpose'].unique()))
plt.title('Distribution of Loan Defaulters by Loan Purpose')
plt.xlabel("Purpose")
plt.ylabel("Number of Loan Defaulters")
# To rotate axis text
plt.xticks(rotation=90)
plt.show()


print (loan[loan['loan_status']=='Charged Off'].groupby(['purpose'])['purpose'].count())

Observation:

1) There are more defaulters from 'debt_consolidation','other', 'credit_card' and 'small_business'. Let's check this at granular level by combining with Home Ownership and by each Grade.

In [ ]:
# Distribution of Loan Defaulters by Home Ownerships & Loan Purpose
plt.figure(figsize=(12,6))
sns.countplot(x='home_ownership', hue='purpose', data=loan[loan['loan_status']=='Charged Off'], palette="Set3")

plt.title('Distribution of Loan Defaulters by Home Ownerships & Loan Purpose')
plt.xlabel("Home Ownerships")
plt.ylabel("Number of Loan Defaulters")
plt.tight_layout()
plt.show()

print (loan[loan['loan_status']=='Charged Off'].groupby(['home_ownership','purpose'])['home_ownership'].count())

Observation:

1) From RENT category, there are more defaulters from 'debt_consolidation','other', 'credit_card' and 'small_business'.

2) From MORTGAGE category, there are more defaulters from 'debt_consolidation','home_improvement', 'credit_card' and 'small_business'.

Overall, one should be carefull with 'debt_consolidation', 'credit_card' and 'small_business' loans when the borrowers dont have own house.

In [ ]:
#Distribution of Loan Defaulters by Grade
sns.countplot(x='grade',  data=loan[loan['loan_status']=='Charged Off'], order=np.sort(loan['grade'].unique()))
plt.title('Distribution of Loan Defaulters by Grade')
plt.xlabel("Grade")
plt.ylabel("Number of Loan Defaulters")
plt.show()

print (loan[loan['loan_status']=='Charged Off'].groupby(['grade'])['grade'].count())

Observation:

1) It shows there are more defaulters in B,C and D grades.

2) Grades F,G(more intereste rate grades) are having less defaulters which is a good indicator.

Let's check it in granular level.

In [ ]:
# Distribution of Loan Defaulters by Grade & Loan Purpose
plt.figure(figsize=(12,6))
sns.countplot(x='grade', hue='purpose', data=loan[loan['loan_status']=='Charged Off'], palette="Set3")

plt.title('Distribution of Loan Defaulters by Grade & Loan Purpose')
plt.xlabel("Grade")
plt.ylabel("Number of Loan Defaulters")
plt.tight_layout()
plt.show()

print (loan[loan['loan_status']=='Charged Off'].groupby(['grade','purpose'])['grade'].count())

Observation:

1) From all grades, there are more defaulters from 'debt_consolidation', 'others', 'credit_card' and 'small_business' purpose loans.

# Summary of Bivariate Observations:


# Bivariate Analysis on Continuous Variables:

#Correlation Plot:

a) There were no highly negative correlation between numerical columns.

b) 0.45 correlation between term_months and interest rate(int_rate).

c) 0.47 correlation between revol_util and interest rate(int_rate).

d) No much correlation between Annual Income, DTI with other numerical columns.



#ROI vs Interest rates and Annual income:

a) There was a positive correlation between interest rates and ROI for borrowers who paid their loans, but for those who didn’t pay the ROI was negative and it shows no correlation with interest rates.                                                        
b) Borrowers with higher income has positive ROI while borrowers with lower income has negative ROI.

#Annual Income Vs Interest Rate by each grade:

By charts, it is evident that the interest rate range changes from one grade to other, and also there are few borrowers with near 6% falling in B,C,D and E grade's.

#Annual Income Vs DTI distribution by each grade:

By charts, it shows there is no DTI range constraint from one grade to other similar to interest rate which we have seen above.


# Bivariate Analysis on Categorical Variables:

#ROI Bivariate Distribution plots:

1) ROI vs Grade by loan_status: From fully_paid loan, grades E,F,G have high mean ROI, this is because of high interest rates.

2) ROI vs Loan Purpose by loan_status: 
a) From fully_paid loans, almost all categories have nearby mean ROI.
b) From Charged_off loan, small_business, other and medical have low mean ROI.

3) ROI vs Home Ownership by loan_status: From Charged_off loans, Rent and Mortgage have low mean ROI.

4) ROI vs Income Segment by loan_status: 
a) From fully_paid loans, ROI is bit high for upper middle and upper Income_segment.
b) From Charged_off loan, ROI is very low for lower Income_segment.

#Interest Rate Bivariate Distribution Plots:

1) Interest Rate vs Term Months by loan_status:
a) It is evident that 60 months term loans have more interest rate.
b) There are more defaulters in both 36, 60 month terms because of high interest rates.

2) Interest Rate vs Income Segment by loan_status:
a) By charts, it shows that, at every income segment, the defaulters are due to high interest rates.

#DTI Bivariate Distribution Plots:

1) DTI vs Income Segment by loan_status:
a) DTI is less in lower segment, as the annual income is less and they might not have any loans and hence less DTI. If we have age details then we can predict more with borrower’s age, income and employment length.
b) DTI is high in lower middle and middle segments.
c) DTI is less in upper middle and upper segments, as the income is high they might have financial freedom. Note that there are some borrower’s in upper segment who have DTI more than 25.

# Loan Defaulters Analysis:

1) Distribution of Loan Defaulters by Loan Purpose:
a) There are more defaulters from 'debt_consolidation','other', 'credit_card' and 'small_business'. Let's check this at granular level by combining with Home Ownership and by each Grade.

2) Distribution of Loan Defaulters by Home Ownerships:
a) By charts, it shows there are more defaulters in RENT and MORTGAGE.

3) Distribution of Loan Defaulters by Home Ownerships & Loan Purpose:
a) There are more defaulters with 'debt_consolidation', 'credit_card' and 'small_business' purpose from Rent, Mortgage category and hence, should be carefull when borrowers don't have own house.

4) Distribution of Loan Defaulters by Grade:
a) By charts, it shows there are more defaulters in B,C and D grades.
b) Grades F,G(more intereste rate grades) are having less defaulters which is a good indicator.

5) Distribution of Loan Defaulters by Grade & Loan Purpose:
By charts, from all grades, there are more defaulters from 'debt_consolidation', 'others', 'credit_card' and 'small_business' purpose loans.

# Conclusion:

1) Number of loans issued increased steadily by every year with a slight decrease in 2008.

2) Of settled loans, 83% were Fully Paid and 14% were Charged Off.

3) Borrowers with own house and the purpose of loan with consolidate debt, 'credit_card' and 'small_business' are not at much risk, but borrower with rent,mortgage are high risk applicants.

4) Majority of loans were from A, B, and C grade.

5) There is an inverse relationship between interest rate and loan grade - lower grades(E,F,G) have higher interest rate.

6) From ROI analysis, it shows that borrowers with the best credit profiles or the lowest loan amounts will not end up being the most profitable and the borrowers who seemed to have worst credit indicators ended up being more profitable from E,F,G grades.

7) Overall, there are more defaulters from 'debt_consolidation', 'others', 'credit_card' and 'small_business' purpose loans from all grades.